<a href="https://colab.research.google.com/github/panpapap/Portfolio/blob/master/SQL_Project_World_Population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

This project aims to explore the database from CIA Factbook Data, which contains information about all of the countries on Earth. We will address the following questions:

* Which country has the highest population?

* Which country has the least number of people?

* Which country is witnessing the highest population growth?

* Which is the most densely populated country in the world?

* Which countries are above average in population and below average in area?

**SQL skills used**: AVG, ORDER BY, ROUND, WHERE, LIMIT, MIN, MAX, subqueries


https://www.cia.gov/the-world-factbook/


In [ ]:
#import sql3, pandas and connect to the databse.
import sqlite3
import pandas as pd
conn = sqlite3.connect("/content/drive/MyDrive/Portfolio_files/SQL/factbook.db")

#activates the cursor
cursor = conn.cursor()

#the SQL query to look at the tables in the databse
q = "SELECT * FROM sqlite_master WHERE type='table';"

#execute the query and read it in pandas, this returns a table in pandas form
db = pd.read_sql_query(q, conn)
db

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY..."


In [ ]:
q1 = "SELECT * FROM facts"

dt = pd.read_sql_query(q1, conn)

dt.head()

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342.0,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3029278.0,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,39542166.0,1.84,23.67,4.31,0.92
3,4,an,Andorra,468.0,468.0,0.0,85580.0,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700.0,1246700.0,0.0,19625353.0,2.78,38.78,11.49,0.46


In [ ]:
#First let's look at some min and max number to find any outlier
q2 = '''
SELECT MIN(Population) min_population, MAX(Population) max_population,
MIN(population_growth) min_pop_growth, MAX(population_growth) max_pop_growth
FROM facts
'''
dt = pd.read_sql_query(q2, conn)
dt

,min_population,max_population,min_pop_growth,max_pop_growth
0,0,7256490011,0.0,4.02


It seems like we have a world total population row and a country with 0 population. Let's find out the name of those outliers and exclude them from the further analysis.

In [ ]:
q3 = '''
SELECT name, population FROM facts
WHERE population = 0 OR population > 7000000000
'''
dt = pd.read_sql_query(q3, conn)
dt

,name,population
0,Antarctica,0
1,World,7256490011


In [ ]:
#Which country has the highest population?
q4 = '''SELECT name, max(population)
        FROM facts
        WHERE name <> "World"'''
dt = pd.read_sql_query(q4,conn)
dt

,name,max(population)
0,China,1367485388


In [ ]:
#Which country has the lowest population?
q5 = '''SELECT name, min(population)
        FROM facts
        WHERE name <> "Antarctica"'''
dt = pd.read_sql_query(q5,conn)
dt

,name,min(population)
0,Pitcairn Islands,48


From the result, we see that Chinese has the highest population at 1,367,485,388 and there are only 48 people live in Pitcairn Islands. Note that we exclude World total population and Antarcitca which has no population out of this analysis.

In [ ]:
#Which country is witnessing the highest population growth?
q6 = '''SELECT name, max(population_growth)
        FROM facts'''
dt = pd.read_sql_query(q6,conn)
dt

,name,max(population_growth)
0,South Sudan,4.02


South Sudan has the highest growth of 4.02%

In [ ]:
#Which is the most densely populated country in the world?
q5 = '''SELECT name, ROUND(CAST(population as float)/CAST(area as float),2) as density
        FROM facts
        ORDER BY density DESC
        LIMIT 5'''
dt = pd.read_sql_query(q5,conn)
dt

,name,density
0,Macau,21168.96
1,Monaco,15267.50
2,Singapore,8141.28
3,Hong Kong,6445.04
4,Gaza Strip,5191.82


Macau has the highest population density in the world, at 21168.96.

In [ ]:
# What is the average number of population?
q6 = '''SELECT ROUND(AVG(population),0), ROUND(AVG(area),2)
        FROM facts
        WHERE name <> "World" AND name <> "Antarctica"'''
dt = pd.read_sql_query(q6,conn)
dt

,"ROUND(AVG(population),0)","ROUND(AVG(area),2)"
0,32377011.0,555093.55


The average number of population in the world is 62,352,584 and the average area is 555,093.55 km2.

Now we will find out how many countries have more than average number of population but under average area.

In [ ]:
#Which countries are above average population and below average area?
q7 = '''SELECT name, population, area, ROUND(CAST(population as float)/CAST(area as float),2) as density
        FROM facts
        WHERE population > (SELECT AVG(population) FROM facts) AND area < (SELECT AVG(area) FROM facts)
        AND name <> "World"
        ORDER BY density DESC'''

dt = pd.read_sql_query(q7,conn)
dt

,name,population,area,density
0,Bangladesh,168957745,148460,1138.07
1,Philippines,100998376,300000,336.66
2,Japan,126919659,377915,335.84
3,Vietnam,94348835,331210,284.86
4,United Kingdom,64088222,243610,263.08
5,Germany,80854408,357022,226.47
6,Thailand,67976405,513120,132.48
